In [ ]:
import sys
sys.path.append("./models")
import random
from schnapsen.bots import RandBot
from schnapsen.bots import RdeepBot

from ML_bot import create_replay_memory_dataset

rng = random.Random(42)

bot1 = RandBot(rng)
bot2 = RdeepBot(num_samples=16, depth=4, rand=rng)

create_replay_memory_dataset(bot1=bot1, bot2=bot2)


In [ ]:
import sys
sys.path.append("./models")

from ML_bot import train_model

train_model("NN")

In [ ]:
import pathlib
import random

from schnapsen.game import SchnapsenGamePlayEngine
from schnapsen.bots import RandBot

from ML_bot import MLPlayingBot

engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")

winner_id, game_points, score = engine.play_game(bot1, bot2, random.Random(0))
print(f"Game ended. Winner is {winner_id._Bot__name} with {game_points} points and {score}")

In [2]:
import sys
sys.path.append("./models")
from ML_bot import MLPlayingBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import RandBot
from Deepbot import DeepLearningBot
from schnapsen.game import SchnapsenGamePlayEngine
import pathlib
import random


engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

model_path = "./models/schnapsen_model.pth"

rng = random.Random(42)

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")
bot3 = RdeepBot(num_samples=5, depth=2, rand=rng, name="RdeepBot")
bot4 = DeepLearningBot(model_path=model_path, input_size=173, hidden_size=64, name="DLbot")

def round_robin_tournament():
    wins_MLBot = 0
    wins_RDeep = 0
    wins_Randy = 0
    for i in range (1000):

        if i % 500 == 0:
            print(f"Current progress: game {i}")

        winner_id, game_points, score = engine.play_game(bot1, bot4, random.Random(i))

        if winner_id._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id2, game_points2, score2 = engine.play_game(bot1, bot3, random.Random(i))

        if winner_id2._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id2._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id3, game_points3, score3 = engine.play_game(bot4, bot3, random.Random(i))

        if winner_id3._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id3._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1



    print (f"DLBot: {wins_MLBot}")
    print (f"Randy: {wins_Randy}")
    print (f"Rdeep: {wins_RDeep}")

round_robin_tournament()

/home/roti/workspace/uni_projects/cardbot/models/Deepbot.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_location=

current progress: game 0
current progress: game 500
DLBot: 675
Randy: 1102
Rdeep: 1223


In [1]:
import sys
sys.path.append("./models")
from Deepbot import train_DL_model

data_file = "./ML_replay_memories/random_random_10k_games.txt"  # Replace with your data file path
output_model_path = "./models/schnapsen_model.pth"
input_dim = 173
hidden_dim = 64
train_DL_model(data_file, output_model_path, input_dim, hidden_dim)

Epoch 1/50, Train Loss: 0.4942, Validation Loss: 0.4779
Epoch 2/50, Train Loss: 0.4700, Validation Loss: 0.4685
Epoch 3/50, Train Loss: 0.4598, Validation Loss: 0.4538
Epoch 4/50, Train Loss: 0.4468, Validation Loss: 0.4466
Epoch 5/50, Train Loss: 0.4317, Validation Loss: 0.4385
Epoch 6/50, Train Loss: 0.4160, Validation Loss: 0.4227
Epoch 7/50, Train Loss: 0.4062, Validation Loss: 0.4160
Epoch 8/50, Train Loss: 0.3991, Validation Loss: 0.4132
Epoch 9/50, Train Loss: 0.3918, Validation Loss: 0.4154
Epoch 10/50, Train Loss: 0.3876, Validation Loss: 0.4104
Epoch 11/50, Train Loss: 0.3813, Validation Loss: 0.4111
Epoch 12/50, Train Loss: 0.3776, Validation Loss: 0.4125
Epoch 13/50, Train Loss: 0.3740, Validation Loss: 0.4098
Epoch 14/50, Train Loss: 0.3699, Validation Loss: 0.4094
Epoch 15/50, Train Loss: 0.3662, Validation Loss: 0.4069
Epoch 16/50, Train Loss: 0.3629, Validation Loss: 0.4110
Epoch 17/50, Train Loss: 0.3600, Validation Loss: 0.4133
Epoch 18/50, Train Loss: 0.3584, Validat